In [1]:
import numpy as np
import pandas as pd
data = pd.read_csv("promise-2_0a-packages-csv/eclipse-metrics-packages-2.0.csv", sep = ";")
data

,plugin,packagename,pre,post,ACD_avg,ACD_max,ACD_sum,FOUT_avg,FOUT_max,FOUT_sum,...,NORM_ParameterizedType,NORM_QualifiedType,NORM_WildcardType,NORM_NormalAnnotation,NORM_MarkerAnnotation,NORM_SingleMemberAnnotation,NORM_MemberValuePair,NORM_AnnotationTypeDeclaration,NORM_AnnotationTypeMemberDeclaration,NORM_Modifier
0,org.eclipse.jdt.junit,org.eclipse.jdt.internal.junit.launcher,5,0,0.571429,4.0,4.0,5.980769,29.0,311.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.017372
1,org.eclipse.jdt.junit,org.eclipse.jdt.internal.junit.runner,2,0,0.000000,0.0,0.0,4.000000,22.0,168.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.051474
2,org.eclipse.jdt.junit,org.eclipse.jdt.internal.junit.ui,9,0,1.238095,14.0,26.0,4.321267,33.0,955.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.034461
3,org.eclipse.jdt.junit,org.eclipse.jdt.internal.junit.util,2,0,0.555556,4.0,5.0,3.752941,80.0,319.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028606
4,org.eclipse.jdt.junit,org.eclipse.jdt.internal.junit.wizards,6,0,2.000000,5.0,18.0,6.552632,63.0,996.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,org.eclipse.help.ui,org.eclipse.help.ui.internal.browser.hpux,0,0,0.000000,0.0,0.0,1.750000,11.0,28.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.059289
373,org.eclipse.help.ui,org.eclipse.help.ui.internal.browser.linux,0,0,0.000000,0.0,0.0,2.764706,19.0,47.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.053191
374,org.eclipse.help.ui,org.eclipse.help.ui.internal.browser.solaris,0,0,0.000000,0.0,0.0,2.000000,10.0,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.061420
375,org.eclipse.help.ui,org.eclipse.help.ui.internal.search,3,1,1.600000,5.0,8.0,6.700000,61.0,201.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027916


In [11]:
data_drop = data.drop("post",axis = 1)

In [56]:
def gini(x,y):
    gini_num = (x*y)/((x+y)**2)
    return gini_num

In [90]:
class treenode:
    def __init__(self, sample):
        
        self.sample = sample
        
        num_sample = len(sample)
        num_sample_0 = len(sample[sample["post"] == 0])
        num_sample_1 = num_sample - num_sample_0
        self.sample_class_0 = num_sample_0
        self.sample_class_1 = num_sample_1
        if num_sample_0 > num_sample_1:
            sample_predict = 0
        else:
            sample_predict = 1
        self.sample_predict = sample_predict

        self.gini = gini(num_sample_0 , num_sample_1)

        self.type = "root"
        self.leftleaf = ""
        self.rightleaf = ""
        self.split_feature = ""
        self.c = 0
    
    def node_split(self, nmin, minleaf, nfeat):
        sample = self.sample
        #sample_x = sample.drop("post", axis = 1)
        #sample_y = sample["post"]
        columns = sample.shape[1] - 1
        features_choice = np.random.choice(columns, nfeat, replace = False)
        
        gini_min = 1
        feature_choice = None
        c_choice = 0
        

        for feature_num in features_choice:
            feature = sample.columns[feature_num]
            values = [float(i) for i in list(sample[feature].unique())]
            thresholds = []
            for i in range(len(values)-1):
                thresholds.append((values[i] + values[i+1])/2)
                
            for thresh in thresholds:
                left = sample[sample[feature] >= thresh]
                right = sample[sample[feature] < thresh]
                x_left = left.drop("post", axis = 1)
                y_left = left["post"]
                x_right = right.drop("post", axis = 1)
                y_right = right["post"]
                if len(y_left) < minleaf or len(y_right) < minleaf:
                    continue
                if len(y_left) == 0 or len(y_right) == 0:
                    continue
                gini_num = gini(len(x_left),len(x_right))
                if gini_num < gini_min:
                    feature_choice = feature
                    c_choice = thresh
                
        split_info = [gini_min, feature_choice, c_choice]
        
        return split_info

In [131]:
def tree_grow(x, y, nmin, minleaf, nfeat):
    x["post"] = y
    sample = x
        
    root_node = treenode(sample)
    
    if len(y) < nmin or len(y.unique()) == 1:
        root_node.type = "leaf"
        return root_node
    
    split_info = root_node.node_split(nmin, nfeat, minleaf)
    root_node.split_feature = split_info[1]
    root_node.c = split_info[2]
    
    if split_info[1] is None:
        root_node.type = "leaf"
        return root_node
    
    left = sample[sample[root_node.split_feature] >= root_node.c]
    right = sample[sample[root_node.split_feature] < root_node.c]
    x_left = left.drop("post", axis = 1)
    y_left = left["post"]
    x_right = right.drop("post", axis = 1)
    y_right = right["post"]
    
    if len(y_left) < minleaf or len(y_right) < minleaf:
        return root_node
    else:
        root_node.leftleaf = tree_grow(x_left, y_left, nmin, minleaf, nfeat)
        root_node.rightleaf = tree_grow(x_right, y_right, nmin, minleaf, nfeat)

    return root_node
    
        
            
        
def tree_pred(x, tr):

    if tr.type == "leaf":
        return tr.sample_predict

    else:
        if x[tr.split_feature] >= tr.c:
            return tree_pred(x, tr.leftleaf)
        else:
            return tree_pred(x, tr.rightleaf)

In [123]:
x = data.drop(["post", "plugin", "packagename"], axis = 1)
y = data["post"]

tree = tree_grow(x, y, 15, 5, 41)

In [124]:
data_test = pd.read_csv("promise-2_0a-packages-csv/eclipse-metrics-packages-3.0.csv", sep = ";")
data_test

,plugin,packagename,pre,post,ACD_avg,ACD_max,ACD_sum,FOUT_avg,FOUT_max,FOUT_sum,...,NORM_ParameterizedType,NORM_QualifiedType,NORM_WildcardType,NORM_NormalAnnotation,NORM_MarkerAnnotation,NORM_SingleMemberAnnotation,NORM_MemberValuePair,NORM_AnnotationTypeDeclaration,NORM_AnnotationTypeMemberDeclaration,NORM_Modifier
0,org.eclipse.jdt.junit,org.eclipse.jdt.internal.junit.launcher,11,2,1.222222,11.0,11.0,6.818182,39.0,525.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.018873
1,org.eclipse.jdt.junit,org.eclipse.jdt.internal.junit.refactoring,0,1,0.000000,0.0,0.0,2.545455,17.0,56.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025129
2,org.eclipse.jdt.junit,org.eclipse.jdt.internal.junit.ui,23,7,1.416667,14.0,51.0,4.300439,56.0,1961.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033287
3,org.eclipse.jdt.junit,org.eclipse.jdt.internal.junit.util,5,1,0.600000,4.0,6.0,4.041237,80.0,392.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028996
4,org.eclipse.jdt.junit,org.eclipse.jdt.internal.junit.wizards,5,0,1.800000,5.0,18.0,6.455090,64.0,1078.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,org.eclipse.team.ui,org.eclipse.team.internal.ui.synchronize,69,11,1.511111,10.0,68.0,3.610927,64.0,2181.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035323
657,org.eclipse.team.ui,org.eclipse.team.internal.ui.synchronize.actions,23,6,1.000000,3.0,18.0,4.811966,31.0,563.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029950
658,org.eclipse.team.ui,org.eclipse.team.internal.ui.wizards,11,2,1.090909,4.0,12.0,5.351064,71.0,503.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022291
659,org.eclipse.team.ui,org.eclipse.team.ui,6,1,0.250000,1.0,2.0,1.270270,16.0,47.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.064309


In [135]:
x = data_test.drop(["post", "plugin", "packagename"], axis = 1)
y_test = data_test["post"]
x

,pre,ACD_avg,ACD_max,ACD_sum,FOUT_avg,FOUT_max,FOUT_sum,MLOC_avg,MLOC_max,MLOC_sum,...,NORM_ParameterizedType,NORM_QualifiedType,NORM_WildcardType,NORM_NormalAnnotation,NORM_MarkerAnnotation,NORM_SingleMemberAnnotation,NORM_MemberValuePair,NORM_AnnotationTypeDeclaration,NORM_AnnotationTypeMemberDeclaration,NORM_Modifier
0,11,1.222222,11.0,11.0,6.818182,39.0,525.0,10.935065,61.0,842.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.018873
1,0,0.000000,0.0,0.0,2.545455,17.0,56.0,4.363636,19.0,96.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.025129
2,23,1.416667,14.0,51.0,4.300439,56.0,1961.0,7.015351,75.0,3199.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033287
3,5,0.600000,4.0,6.0,4.041237,80.0,392.0,6.814433,118.0,661.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.028996
4,5,1.800000,5.0,18.0,6.455090,64.0,1078.0,10.365269,63.0,1731.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.024622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,69,1.511111,10.0,68.0,3.610927,64.0,2181.0,6.402318,105.0,3867.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.035323
657,23,1.000000,3.0,18.0,4.811966,31.0,563.0,7.683761,36.0,899.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029950
658,11,1.090909,4.0,12.0,5.351064,71.0,503.0,8.808511,125.0,828.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.022291
659,6,0.250000,1.0,2.0,1.270270,16.0,47.0,2.567568,14.0,95.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.064309


In [136]:
y_pred = []
for i in range(len(x)):
    row = x.iloc[i]
    result = tree_pred(row, tree)
    y_pred.append(result)

In [137]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_test = [1 if i != 0 else 0 for i in y_test]

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

precision = precision_score(y_test, y_pred, average='binary')
print(f"Precision: {precision:.2f}")

recall = recall_score(y_test, y_pred, average='binary')
print(f"Recall: {recall:.2f}")

Accuracy: 0.69
Precision: 0.70
Recall: 0.62
